In [ ]:
import pandas as pd 
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import json

In [ ]:
#Browser setup
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome")

In [ ]:
base_url = "https://health.usnews.com/best-hospitals/rankings/cancer?page="
mini_url = "https://health.usnews.com"

In [ ]:
soup = bs(browser.html, 'lxml')

In [ ]:
list_ = [str(x) for x in list(range(1,10))]

In [ ]:
for x in list_:
    print(base_url + x)

In [ ]:
df = pd.DataFrame([])
def dataframes(url):
    city = []
    browser.visit(url)
    soup = bs(browser.html, 'lxml')
    name = soup.find_all("h3", class_="block-flush heading-large")
    ranking = soup.find_all("span", class_="heading-small")
    score = soup.find_all("div", class_="text-strong text-tighter")
    web = soup.find_all(class_="search-match-card-view")
    
    names = [x.get_text(strip=True) for x in name]
    rankings = [x.get_text(strip=True) for x in ranking]
    scores = [x.get_text(strip=True) for x in score]
    info = soup.find_all("div", class_="block-tight")
    websites = [mini_url + x.a["href"] for x in web]
    
    for x in range(0,len(info[1:-1]),2):
        city.append(info[x].get_text(strip=True).split("|")[-1])
        
    df_names = pd.DataFrame(names)
    df_rankings = pd.DataFrame(rankings)
    df_scores = pd.DataFrame(scores)
    df_cities = pd.DataFrame(city[1:])
    df_web = pd.DataFrame(websites)
    
    df_1 = pd.merge(df_names, df_rankings, left_index=True, right_index=True)
    df_2 = pd.merge(df_1, df_scores, left_index=True, right_index=True)
    df_3 = pd.merge(df_2, df_cities, left_index=True, right_index=True)
    df = pd.merge(df_3, df_web, left_index=True, right_index=True)
    df.columns = ["name", "rank", "score", "cities", "websites"]
    return df

In [ ]:
p1 = dataframes("https://health.usnews.com/best-hospitals/rankings/cancer?page=1")
p2 = dataframes("https://health.usnews.com/best-hospitals/rankings/cancer?page=2")
p3 = dataframes("https://health.usnews.com/best-hospitals/rankings/cancer?page=3")
p4 = dataframes("https://health.usnews.com/best-hospitals/rankings/cancer?page=4")
p5 = dataframes("https://health.usnews.com/best-hospitals/rankings/cancer?page=5")
p6 = dataframes("https://health.usnews.com/best-hospitals/rankings/cancer?page=6")
p7 = dataframes("https://health.usnews.com/best-hospitals/rankings/cancer?page=7")
p8 = dataframes("https://health.usnews.com/best-hospitals/rankings/cancer?page=8")
p9 = dataframes("https://health.usnews.com/best-hospitals/rankings/cancer?page=9")

In [ ]:
m1 = p1.append(p2, ignore_index=True)
m2 = m1.append(p3, ignore_index=True)
m3 = m2.append(p4, ignore_index=True)
m4 = m3.append(p5, ignore_index=True)
m5 = m4.append(p6, ignore_index=True)
m6 = m5.append(p7, ignore_index=True)
m7 = m6.append(p8, ignore_index=True)
df = m7.append(p9, ignore_index=True)

In [ ]:
df["num_rank"] = df.iloc[:,1].map(lambda x : x.split(' ')[0].split("#")[-1])
df["num_score"] = df.iloc[:,2].map(lambda x : x.split("/")[0])
df['city_name'] = df.iloc[:,3].map(lambda x : x.split(',')[0])
df['state'] = df.iloc[:,3].map(lambda x : x.split(',')[1].split(' ')[1])
df['zipcode'] = df.iloc[:,3].map(lambda x : x.split(',')[1].split(' ')[2])
df["num_rank"] = df["num_rank"].replace("High", 0).astype(int)
df["num_score"] = df["num_score"].astype(float)

In [ ]:
lat = []
lng = []

names = df['name'].tolist()

for name in names:
    target_city = name
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_city, gkey)

    geo_data = requests.get(target_url).json()
    lat.append(geo_data['results'][0]['geometry']['viewport']['northeast']['lat'])
    lng.append(geo_data['results'][0]['geometry']['viewport']['northeast']['lng'])
    

In [ ]:
df['lat'] = pd.DataFrame(lat)
df['lng'] = pd.DataFrame(lng)

In [ ]:
df.to_csv("../cancer_graphical_data/data/hospital_ranking.csv", index=False)

In [ ]:
df.head()